In [32]:
import sys
import numpy
import copy
import math

In [10]:
# enumerate binary trees of size n
# the standard form equations are B = U + V, U = z, V = B * B
# intermediate values are tabulated by a dict k -> (b_k, u_k, v_k)

idxB = 0
idxU = 1
idxV = 2

def tabulate(k, table):
    v_k = 0
    if k != 0:
        for i in range(1,k):
            v_k += table[i][idxB] * table[k-i][idxB]
    u_k = 1 if k == 1 else 0
    b_k = u_k + v_k
    table[k] = (b_k, u_k, v_k)
    return table
    
def BinaryTree(n):
    t = {}
    for k in range(n+1):
        t = tabulate(k, t)
    print t[n][idxB]
    return

for k in range(0,20):
    BinaryTree(k)

0
1
1
2
5
14
42
132
429
1430
4862
16796
58786
208012
742900
2674440
9694845
35357670
129644790
477638700


In [41]:
# enumerate combinatorial structures from standard form
# supports union, cartesian product and Atom rules
# intermediate values are tabulated by a dict k -> (dict r -> r_k)

class Union:
    def __init__(self, *args): # args[0] = args[1] + args[2]
        self.Type = 'Union'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Union) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)

class Product:
    def __init__(self, *args): # args[0] = args[1] * args[2]
        self.Type = 'Product'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Product) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)
            
class Sequence:
    def __init__(self, *args): # args[0] = seq(args[1])
        self.Type = 'Sequence'
        if len(args) == 2:
            self.Value = args[0]
            self.SubRule = args[1]
        elif len(args) == 1:
            self.SubRule = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Sequence) and \
            self.SubRule == other.SubRule
        
class Multiset:
    def __init__(self, *args): # args[0] = set(args[1])
        self.Type = 'Multiset'
        if len(args) == 2:
            self.Value = args[0]
            self.SubRule = args[1]
        elif len(args) == 1:
            self.SubRule = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Multiset) and \
            self.SubRule == other.SubRule
        
class Theta:
    def __init__(self, *args): # args[0] = theta(args[1])
        self.Type = 'Theta'
        if len(args) == 2:
            self.Value = args[0]
            self.SubRule = args[1]
        elif len(args) == 1:
            self.SubRule = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Theta) and \
            self.SubRule == other.SubRule

class Atom:
    def __init__(self, *args):
        self.Type = 'Atom'
        if len(args) == 2:
            self.Value = args[0]
            self.Size = args[1]
        elif len(args) == 1:
            self.Size = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Atom) and self.Size == other.Size
    
# convert a symbolic equation to standard form
# example: B = Z + B * B -> [B = U + V, U = Z, V = B * B]
def ConvertToStandardForm(eq):
    r, _ = convert(eq, {}, 65)
    return r.values()

# helper function for ConvertToStandardForm
def convert(op, rules, v):
    op1, op2 = (op.SubRule1, op.SubRule2)
    val1, val2 = (None, None)
    evalLeft, evalRight = (False, False)
    if isinstance(op1, Product) or isinstance(op1, Union):
        for r in rules.values():
            rSub = copy.deepcopy(r)
            if isinstance(rSub, Union) or isinstance(rSub, Product):
                if rSub.SubRule1 in rules:
                    if isinstance(rules[rSub.SubRule1], Atom):
                        rSub.SubRule1 = rules[rSub.SubRule1]
                if rSub.SubRule2 in rules:
                    if isinstance(rules[rSub.SubRule2], Atom):
                        rSub.SubRule2 = rules[rSub.SubRule2]
            if rSub == op1:
                val1 = r.Value
                break
        if val1 == None: 
            val1 = chr(v)
            v += 1
            evalLeft = True
    elif isinstance(op1, Atom):
        for r in rules.values():
            if r.Size == op1.Size:
                val1 = r.Value
                break
        if val1 == None:
            val1 = chr(v)
            v += 1
            op1.Value = val1
            rules[val1] = op1
    else: val1 = op1
    if isinstance(op2, Product) or isinstance(op2, Union):
        for r in rules.values():
            rSub = copy.deepcopy(r)
            if isinstance(rSub, Union) or isinstance(rSub, Product):
                if rSub.SubRule1 in rules:
                    if isinstance(rules[rSub.SubRule1], Atom):
                        rSub.SubRule1 = rules[rSub.SubRule1]
                if rSub.SubRule2 in rules:
                    if isinstance(rules[rSub.SubRule2], Atom):
                        rSub.SubRule2 = rules[rSub.SubRule2]
            if rSub == op2:
                val2 = r.Value
                break
        if val2 == None: 
            val2 = chr(v)
            v += 1
            evalRight = True
    elif isinstance(op2, Atom):
        for r in rules.values():
            if r.Size == op2.Size:
                val2 = r.Value
                break
        if val2 == None:
            val2 = chr(v)
            v += 1
            rules[val2] = op1
    else: val2 = op2
    op.SubRule1 = val1
    op.SubRule2 = val2
    rules[op.Value] = op # rules += [op]
    if evalLeft:
        op1.Value = val1
        rules, v = convert(op1, rules, v)
    if evalRight:
        op2.Value = val2
        rules, v = convert(op2, rules, v)
    return rules, v

def evaluateRule(rule, k, table, vals):
    if k not in table:
        table[k] = {}
    if len(rule.Value) == 1 and k < vals[rule.Value]:
        table[k][rule.Value] = 0.0
    elif len(rule.Value) == 2 and k < vals[rule.Value[1]]:
        table[k][rule.Value] = 0.0
    elif isinstance(rule, Union):
        table[k][rule.Value] = table[k][rule.SubRule1] + table[k][rule.SubRule2]
    elif isinstance(rule, Product):
        table[k][rule.Value] = 0.0
        for i in range(0,k+1):
            if rule.SubRule1 not in table[i]:
                continue
            if rule.SubRule2 not in table[k-i]:
                continue
            table[k][rule.Value] += table[i][rule.SubRule1] * table[k-i][rule.SubRule2]
    elif isinstance(rule, Sequence):
        table[k][rule.Value] = 1.0 if k == 0 else 0.0
        for i in range(1,k+1):
            table[k][rule.Value] += table[i][rule.SubRule] * table[k-i][rule.Value]
    elif isinstance(rule, Atom):
        table[k][rule.Value] = 1.0 if rule.Size == k else 0.0
    else: raise Exception('Unsupported rule')
    #print k, "->", table
    # for every rule R, tabulate Theta(R) and vice versa
    if len(rule.Value) == 2:
        table[k][rule.Value[1]] = float(table[k][rule.Value]) / k if k > 0 else 0.0
    elif len(rule.Value) == 1:
        table[k]['T' + rule.Value] = k * float(table[k][rule.Value])
    return table

# compute the valuations of a list of rules
# implementation of algorithm on page 28, section 1.4
def computeRuleValuations(rules):
    val = {}
    for r in rules:
        rv = r.Value if len(r.Value) == 1 else r.Value[1]
        if isinstance(r, Atom):
            val[rv] = r.Size
        else:
            val[rv] = sys.maxint
    return valuate(val, rules)

# current method is to sort by decreasing valuation
# and sort all Atom rules to the front of the list
def sortRulesByValuation(rules):
    vals = computeRuleValuations(rules)
    d = vals.items()
    d_sorted = sorted(d, key=lambda x:x[1], reverse=True)
    r_sorted = []
    for (k,v) in d_sorted:
        for r in rules:
            if r.Value == k or (len(r.Value) == 2 and r.Value[1] == k):
                if isinstance(r, Atom):
                    r_sorted = [r] + r_sorted
                else:
                    r_sorted.append(r)
    return r_sorted, vals
    
# helper for computeRuleValuations()
def valuate(v, rules):
    done = True
    for r in rules:
        rv = r.Value if len(r.Value) == 1 else r.Value[1]
        prev = v[rv]
        if isinstance(r, Atom):
            continue
        elif isinstance(r, Union):
            l = r.SubRule1 if len(r.SubRule1) == 1 else r.SubRule1[1]
            r = r.SubRule2 if len(r.SubRule2) == 1 else r.SubRule2[1]
            v[rv] = min(v[l], v[r])
        elif isinstance(r, Product):
            l = r.SubRule1 if len(r.SubRule1) == 1 else r.SubRule1[1]
            r = r.SubRule2 if len(r.SubRule2) == 1 else r.SubRule2[1]
            v[rv] = v[l] + v[r]
        else: raise Exception('Unsupported rule')
        if v[rv] != prev: done = False
    if done: return v
    else: return valuate(v, rules)

# rules should be sorted in order of increasing minimality
def EnumerateFromStandardForm(rules, n, v):
    t = {}
    for k in range(n+1):
        for r in rules:
            t = evaluateRule(r, k, t, v)
    return t

# test on a binary tree
def BTree(n):
    rules = [Union('B', 'U', 'V'), Product('V', 'B', 'B'), Atom('U', 1)]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['B']

# test on a unary-binary tree
def UTree(n):
    rules = [Union('U', 'A', 'B'), Atom('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['U']

# test on plane general trees in standard form
# the idea here is that a standard form should only contain union, product, Atom
# and theta rules, so A = Seq(B) should be expressed as A = 1 + A * B
def StandardFormPlaneTree(n):
    rules = [Atom('U', 0), Product('V', 'A', 'C'), Union('A', 'U', 'V'), Atom('W', 1), Product('C', 'W', 'A')]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['C']

# test on Fibonacci numbers
def Fib(n):
    rules = [Atom('A', 1), Atom('B', 2), Union('C', 'A', 'B'), Sequence('F', 'C')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['F']

# test on Fibonacci numbers in standard form
def StandardFormFib(n):
    rules = [Atom('A', 1), Atom('B', 2), Union('C', 'A', 'B'), Atom('E', 0), \
             Product('D', 'C', 'F'), Union('F', 'E', 'D')]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['F']

# test on general plane trees
def PlaneTree(n):
    rules = [Atom('U', 1), Product('C', 'U', 'V'), Sequence('V', 'C')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['C']

# test on binary words
def BWord(n):
    rules = [Atom('U', 1), Atom('V', 1), Union('Z', 'U', 'V'), Sequence('W', 'Z')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['W']

# test on simple permutations (factorials)
def Perm(n):
    rules = [Atom('U', 0), Atom('V', 1), Product('W', 'V', 'P'), Theta('X', 'W'), Union('P', 'U', 'X')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['P']

# A = Set(Z)
def SimpleSet(n):
    rules = [Scalar('TZ', 1), Product('TA', 'TZ', 'A'), InverseTheta('A', 'TA')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['A']

def NonPlaneTree(n):
    rules = [Scalar('TZ', 1), Atom('Z', 1), Product('B', 'Z', 'TU'), Product('A', 'TZ', 'U'), \
            Union('TT', 'A', 'B'), Product('TU', 'U', 'TT'), Product('T', 'Z', 'U')]
    for r in sortRulesByValuation(rules):
        if isinstance(r, Scalar) or isinstance(r, Atom):
            print str(r.Value) + " -> " + str(r.Size)
        elif isinstance(r, Union):
            print str(r.Value) + " -> " + str(r.SubRule1) + " + " + str(r.SubRule2)
        elif isinstance(r, Product):
            print str(r.Value) + " -> " + str(r.SubRule1) + " * " + str(r.SubRule2)
        else:
            print "error"

'''
U -> Set(T)  [ NOT COMPUTABLE, so "ignored", and understood as U -> 1/n * Theta(U)  ]

T -> Z * U
U -> InverseTheta(Theta(U))
Theta(U) -> U * Theta(T)
Theta(T) -> A + B
A -> Theta(Z) * U
B -> Z * Theta(U)
Theta(Z) -> 1

rules = [Atom('TZ', 0), Atom('Z', 1), Product('B', 'Z', 'TU'), Product('A', 'TZ', 'U'), \
    Union('TT', 'A', 'B'), Product('TU', 'U', 'TT'), InverseTheta('U', 'TU'), Product('T', 'Z', 'U')]

A = Set(Z)
A = InverseTheta(Theta(A))
Theta(A) = A * Theta(Z)
Theta(Z) = 1

InverseTheta('A', 'TA')
Product('TA', 'A', 1)
Atom('TZ', 0)

rules = [Atom('TZ', 0), Product('TA', 'A', 'TZ'), InverseTheta('A', 'TA')]

          TZ          A            TA  
0       1             1             0
1       1                           
2       1             1             2
3       1                           


T = Z * U
U = InverseTheta(TU)
TU = U * TT
TT = A + B
A = TZ * U
B = Z * TU
TZ = Atom(0)
Z = Atom(1)
'''

# test on a symbolic equation
def SymbolicEquation(eq, n):
    rules = ConvertToStandardForm(eq)
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    seq = []
    for i in range(n):
        seq += [tab[i][eq.Value]]
    return seq

def StandardForm(rules, n, sym):
    sortedRules, v = sortRulesByValuation(rules)
    tab = EnumerateFromStandardForm(sortedRules, n, v)
    seq = []
    fac = 1
    for i in range(n):
        fac *= i if i > 0 else 1
        seq += [int(tab[i][sym] * fac)]
    return seq

# runable code goes here
# op = Union('X', Atom(1), Union(Product(Atom(1), 'X'), \
#    Product(Product(Atom(1), 'X'), 'X')))
# op = Union('X', Atom(1), Product('X', 'X'))
#op = Union('T', Atom(1), Product('T', Product('T', 'T')))
#T = z + T^3
#B = z + T^2

#print SymbolicEquation(op, 20)

rules1 = [Product('TC', 'V', 'TZ'), Atom('Z', 1), Union('V', 'E', 'W'), Atom('E', 0), Product('W', 'V', 'Z')]
rules2 = [Union('O', 'Z', 'V'), Atom('Z', 1), Product('TV', 'O', 'TO')]

print StandardForm(rules1, 10, 'C')
print StandardForm(rules2, 10, 'O')

[0, 1, 1, 2, 6, 24, 120, 720, 5040, 40320]
[0, 1, 1, 3, 15, 105, 945, 10395, 135135, 2027025]


In [51]:
# sort rules by valuation - example

rules = [Union('U', 'A', 'B'), Atom('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
rules_sorted = sortRulesByValuation(rules)
for r in rules_sorted:
    print r.Value

A
D
C
B
U


In [ ]:
U = Z + (Z × U) + (U × Z) + (U × Z × U) <- non-empty triangulations of convex polygons
T(z) = z + 0.5 T(z)^2 + 0.5 T(z^2) <- otter trees
E = Z + E ◦ [{Z × Z} + {Z × Z × Z}] <- balanced 2-3 trees
B = 1 + z + (z + z * z)B <- binary strings with no consecutive zeros
T = z + T * T * T <- ternary trees

Questions for Jeremie
Go through equations above:
 - Balanced 2-3 trees (operator)
 - Otter trees (z^2)
 - Binary trees with no consecutive zeroes (1 vs z)
Alternating zeroes in enumeration sequence

In [104]:
# unit tests, this will be expanded as we add functionality

def run_tests():
    if test_binary_tree() and \
    test_unary_binary_tree() and \
    test_convex_triangulation() and \
    test_otter_tree() and \
    test_balanced_tree() and \
    test_restricted_binary_string() and \
    test_ternary_tree():
        print "OK"
    return

def test(name, eq, n, ref):
    seq = SymbolicEquation(eq, n)
    for i in range(n):
        if seq[i] != ref[i]:
            print "> " + name + " failed, output = " + str(seq) + ", expected = " + str(ref)
            return False
    return True
     
def test_binary_tree():
    eq = Union('X', Atom(1), Product('X', 'X'))
    ref = [0, 1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862, 16796, 58786, 208012, 742900, \
           2674440, 9694845, 35357670, 129644790, 477638700]
    n = 20
    return test("test_binary_tree", eq, n, ref)
    
def test_unary_binary_tree():
    eq = Union('X', Atom(1), Union(Product(Atom(1), 'X'), \
                                       Product(Product(Atom(1), 'X'), 'X')))
    ref = [0, 1, 1, 2, 4, 9, 21, 51, 127, 323, 835, 2188, 5798, 15511, 41835, 113634, \
           310572, 853467, 2356779, 6536382]
    n = 20
    return test("test_unary_binary_tree", eq, n, ref)

def test_convex_triangulation():
    return True

def test_otter_tree():
    return True
    
def test_balanced_tree():
    return True

def test_restricted_binary_string():
    return True

def test_ternary_tree():
    return True

In [105]:
run_tests()

OK


In [ ]:
'''
    # a flimsy attempt to implement unlabelled multiset
    # which probably needs to be scrapped and re-done
    elif isinstance(rule, Multiset):
        innerSum = rule.Value + 'Sub'
        table[k][innerSum] = 0
        for i in range(1,k+1):
            if k % i == 0:
                table[k][innerSum] += table[k/i][rule.SubRule] / i
        table[k][rule.Value] = 0
        for i in range(0,k+1):
            table[k][rule.Value] += table[i][innerSum] * table[k-i][rule.Value]
        if k != 0: 
            table[k][rule.Value] /= k
'''